# Post Entity Benchmarking

This notebook benchmarks the performance of a **Monolithic Post** table versus a **Fragmented** approach (Core vs Content vs Stats).

## Trade-offs Tested
1.  **Write Penalty**: Fragmentation requires 3 inserts (Core, Content, Stats) vs 1. This is expected to be ~6x slower due to transaction overhead.
2.  **Read Optimization (Feed Construction)**: Scanning the thin `PostCore` table (just IDs and timestamps) to find the "Latest 20 Posts" should be faster than scanning the wide `PostMonolith` table which includes potentially large text content and other fields inline.

In [ ]:
# Install dependencies (quietly)
# !pip install sqlmodel faker > /dev/null 2>&1

In [ ]:
from sqlmodel import Field, SQLModel, create_engine, Session, select
from faker import Faker
import time
import random
import os
from uuid import UUID, uuid4
from datetime import datetime
from typing import Optional

os.makedirs("temp/db", exist_ok=True)
DATABASE_URL = "sqlite:///temp/db/benchmarking_post.db"
engine = create_engine(DATABASE_URL, echo=False)
fake = Faker()

## 1. Monolithic Architecture

A single table holding everything about a post.

### Schema
```sql
CREATE TABLE postmonolith (
    pid CHAR(36) PRIMARY KEY,
    cid CHAR(36) INDEX,
    uid CHAR(36) INDEX,
    content TEXT,        -- Heavy
    type VARCHAR(50),
    created_at DATETIME INDEX,
    likes INTEGER,
    dislikes INTEGER,
    shares INTEGER,
    views INTEGER,
    location VARCHAR(255)
);
```

In [ ]:
class PostMonolith(SQLModel, table=True):
    pid: UUID = Field(default_factory=uuid4, primary_key=True)
    cid: UUID = Field(index=True)
    uid: UUID = Field(index=True)
    content: str
    type: str = "text"
    created_at: datetime = Field(default_factory=datetime.utcnow, index=True)
    likes: int = 0
    dislikes: int = 0
    shares: int = 0
    views: int = 0
    location: Optional[str] = None

## 2. Fragmented Architecture

Split into:
*   `PostCore`: Extremely thin (PID, CID, UID, Date). Fits many more rows per memory page.
*   `PostContent`: Heavy content, fetched only on demand.
*   `PostStats`: Volatile counters, separates write traffic from read traffic.

### Schema
```sql
CREATE TABLE postcore (
    pid CHAR(36) PRIMARY KEY,
    cid CHAR(36) INDEX,
    uid CHAR(36) INDEX,
    type VARCHAR(50),
    created_at DATETIME INDEX
);

CREATE TABLE postcontent (
    pid CHAR(36) PRIMARY KEY FOREIGN KEY(postcore.pid),
    content TEXT,
    location VARCHAR(255)
);

CREATE TABLE poststats (
    pid CHAR(36) PRIMARY KEY FOREIGN KEY(postcore.pid),
    likes INTEGER,
    dislikes INTEGER,
    shares INTEGER,
    views INTEGER
);
```

In [ ]:
class PostCore(SQLModel, table=True):
    pid: UUID = Field(default_factory=uuid4, primary_key=True)
    cid: UUID = Field(index=True)
    uid: UUID = Field(index=True)
    type: str = "text"
    created_at: datetime = Field(default_factory=datetime.utcnow, index=True)

class PostContent(SQLModel, table=True):
    pid: UUID = Field(primary_key=True, foreign_key="postcore.pid")
    content: str
    location: Optional[str]

class PostStats(SQLModel, table=True):
    pid: UUID = Field(primary_key=True, foreign_key="postcore.pid")
    likes: int = 0
    dislikes: int = 0
    shares: int = 0
    views: int = 0

## 3. Data Generation

In [ ]:
ENTRY_COUNT = 50_000
CLUSTER_IDS = [uuid4() for _ in range(100)]
USER_IDS = [uuid4() for _ in range(1000)]
print(f"Generating {ENTRY_COUNT} posts...")

SQLModel.metadata.drop_all(engine)
SQLModel.metadata.create_all(engine)

posts_data = []
for _ in range(ENTRY_COUNT):
    posts_data.append({
        "cid": random.choice(CLUSTER_IDS),
        "uid": random.choice(USER_IDS),
        "content": fake.text(max_nb_chars=1000), # Simulating larger content
        "created_at": fake.date_time_this_year(),
        "likes": random.randint(0, 1000),
        "views": random.randint(0, 10000)
    })

In [ ]:
print("Benchmarking Insertions (WRITE PENALTY)...")

# Populate Monolith
start_time = time.perf_counter()
with Session(engine) as session:
    batch = []
    for data in posts_data:
        batch.append(PostMonolith(**data))
        if len(batch) >= 1000:
            session.add_all(batch)
            session.commit()
            batch = []
    if batch:
        session.add_all(batch)
        session.commit()
mono_insert_time = time.perf_counter() - start_time

# Populate Fragmented
start_time = time.perf_counter()
with Session(engine) as session:
    for i, data in enumerate(posts_data):
        core = PostCore(cid=data["cid"], uid=data["uid"], created_at=data["created_at"])
        session.add(core)
        session.flush()
        
        content = PostContent(pid=core.pid, content=data["content"])
        stats = PostStats(pid=core.pid, likes=data["likes"], views=data["views"])
        session.add(content)
        session.add(stats)
        
        if i % 1000 == 0:
            session.commit()
    session.commit()
frag_insert_time = time.perf_counter() - start_time

print(f"Monolith Insert Time: {mono_insert_time:.4f}s")
print(f"Fragmented Insert Time: {frag_insert_time:.4f}s")
print(f"RESULT: Fragmentation is {frag_insert_time/mono_insert_time:.1f}x slower to write.")

## 4. Benchmarks

In [ ]:
print("Benchmarking Feed Scan (READ OPTIMIZATION)...")
# Scenario: Get the top 20 latest posts for a cluster (Feed Construction). 
# This scans the Index/Table to find the rows. 
# Monolith must scan rows that include the heavy 'content'.
# Fragmented scans 'PostCore' which is tiny.

target_cid = CLUSTER_IDS[0]

start_time = time.perf_counter()
with Session(engine) as session:
    # Monolith fetch (scan wide rows)
    posts = session.exec(
        select(PostMonolith)
        .where(PostMonolith.cid == target_cid)
        .order_by(PostMonolith.created_at.desc())
        .limit(20)
    ).all()
mono_feed_time = time.perf_counter() - start_time

start_time = time.perf_counter()
with Session(engine) as session:
    # Fragmented fetch (scan narrow rows)
    posts = session.exec(
        select(PostCore)
        .where(PostCore.cid == target_cid)
        .order_by(PostCore.created_at.desc())
        .limit(20)
    ).all()
frag_feed_time = time.perf_counter() - start_time

print(f"Monolith Feed: {mono_feed_time:.6f}s")
print(f"Fragmented Feed: {frag_feed_time:.6f}s")

ratio = mono_feed_time / frag_feed_time if frag_feed_time > 0 else 0
print(f"\nFragmented Feed Scan is {ratio:.2f}x faster than Monolith.")

In [ ]:
print("Benchmarking Stats Update (Like)...")

# Benchmark Interaction (Like a post)
start_time = time.perf_counter()
with Session(engine) as session:
    # Get valid PID
    pid_target = session.exec(select(PostMonolith.pid)).first()
    p = session.exec(select(PostMonolith).where(PostMonolith.pid == pid_target)).first()
    p.likes += 1
    session.add(p)
    session.commit()
mono_like_time = time.perf_counter() - start_time

start_time = time.perf_counter()
with Session(engine) as session:
    # Get valid PID
    pid_target = session.exec(select(PostStats.pid)).first()
    s = session.exec(select(PostStats).where(PostStats.pid == pid_target)).first()
    s.likes += 1
    session.add(s)
    session.commit()
frag_like_time = time.perf_counter() - start_time

print(f"Monolith Like: {mono_like_time:.6f}s")
print(f"Fragmented Like: {frag_like_time:.6f}s")

ratio = mono_like_time / frag_like_time if frag_like_time > 0 else 0
print(f"\nFragmented Like is {ratio:.2f}x faster than Monolith.")